# Install and Import 

In [1]:
# !pip install tensorflow==2.15.0
# !pip install gym==0.29.1
# !pip install keras
# !pip install keras-rl2
# %pip install scikit-learn

In [2]:
# %pip install ipykernel
# %pip install --upgrade nbformat
# %pip install stable-baselines3[extra]
# %pip install gymnasium==0.29.1

In [3]:
# %pip install stable-baselines3 plotly numpy pandas
# %pip install ipywidgets
# %pip install pandas_ta

In [4]:
# !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [5]:
import numpy as np
import pandas as pd
import random
import logging
import math
from model_config import Path
import os
import torch
import fnmatch

In [6]:
import gymnasium as gym
from gymnasium import Env
from gymnasium import spaces
from gymnasium.utils import seeding

In [7]:
from stable_baselines3.common.env_checker import check_env
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import SubprocVecEnv

In [8]:
import decimal
decimal.getcontext().prec = 28  # Increase precision

In [9]:
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # Используем GPU
    print("CUDA доступна. Работаем на GPU.")
else:
    device = torch.device("cpu")   # Используем CPU
    print("CUDA не доступна. Работаем на CPU.")


CUDA доступна. Работаем на GPU.


# Open Data

## Description of changes

Основано на Model 2, Отличия:
- Меняем коэффициент в расчете reward 

**Если цена увеличилась** ($\text{price\_change} > 0$):

  $$ 
  \text{reward} += \text{net\_worth\_change} - \frac{\left| \left| \text{net\_worth\_change} \right| - \left| \text{price\_change} \right| \right|}{1}
  $$

**Если цена уменьшилась** ($\text{price\_change} \leq 0$):

  $$
  \text{reward} += \text{net\_worth\_change} + \frac{\left| \left| \text{net\_worth\_change} \right| - \left| \text{price\_change} \right| \right|}{1}
  $$

## Settings

In [11]:
window_length = 48
nb_steps = 130000

model_num = 7
data_num = 2

seed = 42
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed= seed)

## Open Data

In [12]:
def find_dataset_file(dnum, folder="data"):
    pattern = f"dataset_{dnum}D_*.csv"
    for filename in os.listdir(folder):
        if fnmatch.fnmatch(filename, pattern):
            return os.path.join(folder, filename)
    return None

In [13]:
data_path = find_dataset_file(data_num, folder= Path["processed"])
print(data_path)
df = pd.read_csv(data_path)
df.shape

d:\PythonScripts\RL_for_Trading\data\processed\dataset_2D_Standart_26cols_51tkn_1t.csv


(195467, 26)

In [14]:
zero_close_prices = df[df['Close_orig'] == 0]
print(f"Number of zero 'Close' prices after scaling: {len(zero_close_prices)}")

Number of zero 'Close' prices after scaling: 0


In [15]:
unique_values = df['Asset_ID_encoded'].unique()
unique_values

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50],
      dtype=int64)

In [16]:
value_counts = df['Asset_ID_encoded'].value_counts()
value_counts

Asset_ID_encoded
35    4358
43    4358
3     4357
8     4356
31    4354
22    4353
30    4351
42    4345
46    4344
49    4343
36    4341
7     4336
34    4335
18    4334
0     4334
29    4331
9     4327
28    4305
47    4296
6     4287
37    4282
24    4270
16    4268
44    4251
41    4190
21    4182
19    4176
20    4106
39    4047
4     3922
13    3919
23    3906
38    3879
50    3879
1     3879
2     3868
26    3744
48    3739
25    3720
17    3664
40    3641
32    3521
33    3258
5     3066
45    2903
10    2903
14    2903
15    2305
27    1985
12    1927
11     419
Name: count, dtype: int64

In [17]:
def split_by_asset_ids(df: pd.DataFrame, test_asset_ids: list):
    test_df = df[df['Asset_ID_encoded'].isin(test_asset_ids)]
    train_df = df[~df['Asset_ID_encoded'].isin(test_asset_ids)]
    return train_df, test_df

In [18]:
test_asset_ids = [45, 47, 32, 41, 7, 1, 38, 5, 31, 26]
train_df, test_df = split_by_asset_ids(df = df, test_asset_ids = test_asset_ids)
print(f"Training data shape: {train_df.shape}")
print(f"Testing data shape: {test_df.shape}")

Training data shape: (157299, 26)
Testing data shape: (38168, 26)


# Game Rule

In [19]:
class TradingEnv(gym.Env):
    metadata = {'render.modes': ['human']}

    def __init__(self, df, mode):
        super(TradingEnv, self).__init__()

        # Save data and initialize parameters
        self.df = df.reset_index(drop=True)
        self.total_steps = len(self.df) - 1
        self.window_length = window_length
        self.mode = mode # test or train

        # Find indices where a new asset starts
        self.asset_start_indices = self._find_asset_start_indices()
        print(self.asset_start_indices)

        # Define action space: Continuous action between -1 and 1
        self.action_space = spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)

        # Define observation space
        num_features = len(self.df.columns) - 1 # Вычли Close_orig
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(self.window_length, num_features), dtype=np.float32)

        # Initialize trading parameters
        self.fee_cost = 0.001
        self.initial_balance = 1000  # Starting balance
        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.fut_net_worth = self.initial_balance

        self.reward = 0
        self.current_step = self.window_length
        self.current_price = 0
        self.tokens_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        self.now_token = (self.df.loc[self.current_step, 'Asset_ID_encoded'])
        print(self.now_token)
        self.prev_token = self.now_token

        self.hist = {
            "current_step": [],
            'balance': [],
            'net_worth': [],
            'tokens_held': [],
            "token": [],
            "current_price": [],
            "reward": [],
            "action": [],
            'total_shares_sold': [],
            'total_sales_value': [],
        }

        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(logging.INFO)

        if self.logger.hasHandlers():
            self.logger.handlers.clear()

        log_file = Path["train_log"](model_num, data_num + 1)
        file_handler = logging.FileHandler(log_file)
        file_handler.setLevel(logging.INFO)

        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        file_handler.setFormatter(formatter)

        self.logger.addHandler(file_handler)
        logging.getLogger().handlers = []


    def _find_asset_start_indices(self):
        """
        Find the indices in the DataFrame where a new asset starts.
        """
        asset_ids = self.df['Asset_ID_encoded']
        start_indices = {asset_ids[0]:0}
        for i in range(1, len(asset_ids)):
            if asset_ids[i] != asset_ids[i - 1]:
                start_indices[asset_ids[i]] = i

        return start_indices


    def reset(self, seed = seed, options=None, reset_hist=False):
        super().reset(seed= seed)
        self.logger.info("Environment reset")

        self.balance = self.initial_balance
        self.net_worth = self.initial_balance
        self.fut_net_worth = self.initial_balance
        self.tokens_held = 0
        self.total_shares_sold = 0
        self.total_sales_value = 0

        print(self.now_token, self.current_step)
        keys = list(self.asset_start_indices.keys())
        if len(keys) > (keys.index(self.now_token) + 1) and self.current_step != self.window_length:
            self.now_token = keys[(keys.index(self.now_token) + 1)]
            print(self.now_token, "Change")
        else:
            if self.mode == "train":
                print(self.now_token, "Restart")
                self.now_token = keys[0] 
            else:
                if self.current_step != self.window_length:
                    return None, None

        self.current_step = self.asset_start_indices[self.now_token] + self.window_length
        self.prev_token = self.now_token

        self.logger.info(f"Starting new episode with token {self.now_token} at step {self.current_step}")

        if reset_hist:
            self.hist = {
                "current_step": [],
                'balance': [],
                'net_worth': [],
                'tokens_held': [],
                "token": [],
                "current_price": [],
                "reward": [],
                "action": [],
                'total_shares_sold': [],
                'total_sales_value': [],
            }

        observation = self._next_observation()
        info = {}
        return observation, info


    def _next_observation(self):
        frame = self.df.drop(columns = ['Close_orig']).loc[self.current_step - self.window_length + 1:self.current_step]
        obs = frame.values
        return obs.astype(np.float32)


    def step(self, action):
        self.reward = 0
        terminated = False
        truncated = False
        self.prev_token = self.now_token

        if isinstance(action, (list, np.ndarray)):
            action = action[0]

        self.logger.info(f"Step: {self.current_step}, Action taken: {action}")
        self._take_action(action)

        self.current_step += 1  # Move to the next time step
        
        if self.current_step >= self.total_steps:
            terminated = True
        else:
            self.now_token = self.df.loc[self.current_step, 'Asset_ID_encoded']
            if self.now_token != self.prev_token:
                self.logger.info(f"Token change at step {self.current_step}: {self.prev_token} -> {self.now_token}")
                print(f"Мы были на {self.prev_token} перешли на {self.now_token}")
                self.now_token = self.prev_token
                terminated = True

        if not terminated:
            self.future_price = self.df.loc[self.current_step, 'Close_orig'] 
            self.fut_net_worth = self.balance + self.tokens_held * self.future_price

            if self.fut_net_worth != 0:
                net_worth_change = (self.fut_net_worth * 100 / self.net_worth) - 100
                price_change = (self.future_price * 100 / self.current_price) - 100
                initial_change = self.net_worth / self.initial_balance
                
                if price_change > 0:
                    self.reward += net_worth_change - (abs(abs(net_worth_change) - abs(price_change)) / 1)
                else:
                    self.reward += net_worth_change + (abs(abs(net_worth_change) - abs(price_change)) / 1)

                if self.net_worth > self.initial_balance:
                    self.reward += initial_change
                else:
                    self.reward += -(1 - initial_change)
                
                self.logger.info(f"step: {self.current_price - 1}, net_worth_change: {net_worth_change}, price_change: {price_change}, initial_change: {initial_change}, reward: {self.reward}")

                if self.net_worth < self.initial_balance * 0.5:
                    if self.mode == "train":
                        self.logger.info("Net worth dropped below 50% of initial balance.")
                        terminated = True
            else:
                self.logger.info("fut_net_worth == 0")

        obs = self._next_observation()
        info = {}

        self.logger.info(f"Net worth: {self.net_worth}, Balance: {self.balance}, Reward: {self.reward}")

        self.hist["current_step"].append(self.current_step - 1)
        self.hist["balance"].append(self.balance)
        self.hist["net_worth"].append(self.net_worth)
        self.hist["tokens_held"].append(self.tokens_held)
        self.hist["token"].append(self.now_token)
        self.hist["current_price"].append(self.current_price)
        self.hist["reward"].append(self.reward)
        self.hist["action"].append(action)
        self.hist["total_shares_sold"].append(self.total_shares_sold)
        self.hist["total_sales_value"].append(self.total_sales_value)

        return obs, self.reward, terminated, truncated, info


    def _take_action(self, action):
        """
        Apply the continuous action to the current state.
        """
        self.current_price = self.df.loc[self.current_step, 'Close_orig']

        action = float(np.clip(action, -1, 1))

        if action < 0:
            proportion = -action  # Convert to positive
            shares_to_sell = int(self.tokens_held * proportion)
            self._sell(shares_to_sell)

        elif action > 0:
            proportion = action
            self._buy(proportion)
        else:
            self.reward += -1
            pass  

        self.net_worth = self.balance + self.tokens_held * self.current_price


    def _buy(self, proportion):
        amount_to_spend = self.balance * proportion

        shares_to_buy = int(amount_to_spend / (self.current_price * (1 + self.fee_cost)))

        if shares_to_buy > 0:
            total_cost = shares_to_buy * self.current_price
            transaction_cost = total_cost * self.fee_cost
            total_cost += transaction_cost

            self.balance -= total_cost
            self.tokens_held += shares_to_buy

            self.logger.info(f"Bought {shares_to_buy} shares at price {self.current_price}")
            self.logger.info(f"Total cost: {total_cost}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.logger.info("Not enough balance to buy.")


    def _sell(self, shares_to_sell):
        if shares_to_sell > self.tokens_held:
            shares_to_sell = self.tokens_held  # Can't sell more than held

        if shares_to_sell > 0:
            total_sale = shares_to_sell * self.current_price
            transaction_cost = total_sale * self.fee_cost
            total_sale -= transaction_cost

            self.balance += total_sale
            self.tokens_held -= shares_to_sell
            self.total_shares_sold += shares_to_sell
            self.total_sales_value += total_sale

            self.logger.info(f"Sold {shares_to_sell} shares at price {self.current_price}")
            self.logger.info(f"Total sale: {total_sale}, Transaction cost: {transaction_cost}")
        else:
            self.reward += -5
            self.logger.info("No shares to sell.")


    def render(self, mode='human', close=False):
        profit = self.net_worth - self.initial_balance
        print(f'Step: {self.current_step}')
        print(f'Balance: {self.balance:.2f}')
        print(f'Shares held: {self.tokens_held}')
        print(f'Net worth: {self.net_worth:.2f}')
        print(f'Profit: {profit:.2f}')

In [20]:
train_env = TradingEnv(train_df, mode = "train")

{0: 0, 2: 4334, 3: 8202, 4: 12559, 6: 16481, 8: 20768, 9: 25124, 10: 29451, 11: 32354, 12: 32773, 13: 34700, 14: 38619, 15: 41522, 16: 43827, 17: 48095, 18: 51759, 19: 56093, 20: 60269, 21: 64375, 22: 68557, 23: 72910, 24: 76816, 25: 81086, 27: 84806, 28: 86791, 29: 91096, 30: 95427, 33: 99778, 34: 103036, 35: 107371, 36: 111729, 37: 116070, 39: 120352, 40: 124399, 42: 128040, 43: 132385, 44: 136743, 46: 140994, 48: 145338, 49: 149077, 50: 153420}
0


In [21]:
test_env = TradingEnv(test_df, mode = "test")

{1: 0, 5: 3879, 7: 6945, 26: 11281, 31: 15025, 32: 19379, 38: 22900, 41: 26779, 45: 30969, 47: 33872}
1


In [22]:
# check_env(train_env)

# Visualisaton Functions

In [23]:
def plot_close_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id]
    
    # Построение графика Close к индексу DataFrame
    fig = px.line(asset_data, x=asset_data.index, y='Close_orig', 
                  title=f'Close Price for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Close': 'Close Price'})
    
    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "close_by_asset"))

In [24]:
def plot_reward_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['reward'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['reward'], mode='lines', name='Reward'))

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Reward = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Reward and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Reward')

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "reward_by_asset"))

In [25]:
def plot_price_change_by_asset(df, asset_id):
    # Фильтрация данных по Asset_ID_encoded
    asset_data = df[df['Asset_ID_encoded'] == asset_id].copy()
    
    # Вычисление процентного изменения цены (Close)
    asset_data['Price_Change_Percent'] = asset_data['Close_orig'].pct_change() * 100
    
    # Построение графика изменения цены в процентах
    fig = px.line(asset_data, x=asset_data.index, y='Price_Change_Percent', 
                  title=f'Price Change Percentage for Asset ID {asset_id}', 
                  labels={'index': 'Index', 'Price_Change_Percent': 'Price Change (%)'})
    
    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "price_change_by_asset"))

In [26]:
def plot_token_data(df, token):
    # Фильтрация данных по выбранному токену
    token_data = df[df['token'] == token]
    
    # Вычисление среднего значения net_worth для данного токена
    avg_net_worth = token_data['net_worth'].mean()

    # Создание графика
    fig = go.Figure()

    # Линия net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['net_worth'], mode='lines', name='Net Worth'))

    # Горизонтальная линия для net_worth = 1000
    fig.add_hline(y=1000, line_color="green", name='Net Worth = 1000')

    # Горизонтальная линия для среднего значения net_worth
    fig.add_hline(y=avg_net_worth, line_color="red", name=f'Average Net Worth = {avg_net_worth:.2f}')

    # Настройка заголовков и осей
    fig.update_layout(title=f'Net Worth and Average for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Net Worth')

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "token_data"))

In [27]:
def plot_action_counts(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token]
    
    # Подсчет количества каждого уникального действия для данного токена
    action_counts = token_data['action'].value_counts().reset_index()
    action_counts.columns = ['action', 'count']

    # Построение бар-чарта для отображения количества каждого действия
    fig = px.bar(action_counts, x='action', y='count', title=f'Count of Actions for {token}', labels={'action': 'Action', 'count': 'Count'})

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "action_counts"))

In [28]:
def plot_relative_change_by_token(df, token):
    # Фильтрация данных по токену
    token_data = df[df['token'] == token].copy()

    # Вычисление относительного изменения для current_price и net_worth
    token_data['Price_Change_Percent'] = token_data['current_price'].pct_change() * 100
    token_data['NetWorth_Change_Percent'] = token_data['net_worth'].pct_change() * 100

    # Создание графика
    fig = go.Figure()

    # Линия для изменения current_price
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['Price_Change_Percent'],
                             mode='lines', name='Current Price Change (%)'))

    # Линия для изменения net_worth
    fig.add_trace(go.Scatter(x=token_data['current_step'], y=token_data['NetWorth_Change_Percent'],
                             mode='lines', name='Net Worth Change (%)'))

    # Настройка заголовков и осей
    fig.update_layout(title=f'Relative Change of Current Price and Net Worth for {token}',
                      xaxis_title='Current Step',
                      yaxis_title='Change (%)')

    # Показать график
    fig.show()
    fig.write_html(Path["plots"](model_num, data_num, "relative_change_by_token"))

# Train Model

## Settings

In [29]:
model = SAC(
    policy='MlpPolicy',  # Use a Multi-Layer Perceptron policy
    env=train_env,
    verbose=1,
    learning_rate=1e-4,  # Adjust learning rate if needed
    batch_size=128,      # Adjust batch size if needed
    tensorboard_log="./sac_tensorboard/",  # Directory for TensorBoard logs
    seed= seed,
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [30]:
# Set up the evaluation callback
from stable_baselines3.common.callbacks import EvalCallback
eval_callback = EvalCallback(
    test_env,                         # Evaluation environment
    best_model_save_path= Path["model_save"](model_num, data_num),   # Directory to save the best model
    log_path= Path["train_log"](model_num, data_num),               # Directory to save evaluation logs
    eval_freq=5000,                   # Evaluate every 5000 steps
    n_eval_episodes=3,                # Number of episodes to evaluate
    deterministic=True,               # Use deterministic actions during evaluation
    render=False                      # Disable rendering during evaluation
)

## Train

In [ ]:
model.learn(
    total_timesteps=nb_steps,
    log_interval=1000,        
    # callback=eval_callback   
    progress_bar= True,
)

0 48
0 Restart
Logging to ./sac_tensorboard/SAC_36


Output()

0 2455

2 Change

2 6551

...

Мы были на 42 перешли на 43

42 132385

43 Change

43 133755

44 Change

In [32]:
model.save(Path["model_save"](model_num, data_num))

In [ ]:
train_hist = train_env.hist
train_hist_df = pd.DataFrame(train_hist)
print(len(train_hist["action"]))

In [ ]:
unique_values = train_hist_df['token'].unique()
unique_values

## Plots

In [ ]:
token = 11
plot_token_data(df = train_hist_df, token = token)
plot_close_by_asset(df= train_df, asset_id= token)
plot_price_change_by_asset(df= train_df, asset_id= token)
plot_relative_change_by_token(df = train_hist_df, token = token)

In [ ]:
train_hist_df.describe()

In [ ]:
train_hist_df

# Test Model

## Test

In [29]:
model = SAC.load(Path["model_save"](model_num, data_num))

In [30]:
obs, info = test_env.reset(reset_hist=True)  # Reset hist at the beginning
for _ in range(len(test_df)):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = test_env.step(action)

    if terminated or truncated:
        obs, info = test_env.reset(reset_hist=False)  # Do not reset hist
        if info is None:
            break

1 48
Мы были на 1 перешли на 5
1 3879
5 Change
Мы были на 5 перешли на 7
5 6945
7 Change
Мы были на 7 перешли на 26
7 11281
26 Change
Мы были на 26 перешли на 31
26 15025
31 Change
Мы были на 31 перешли на 32
31 19379
32 Change
Мы были на 32 перешли на 38
32 22900
38 Change
Мы были на 38 перешли на 41
38 26779
41 Change
Мы были на 41 перешли на 45
41 30969
45 Change
Мы были на 45 перешли на 47
45 33872
47 Change
47 38167


In [31]:
test_hist = test_env.hist
test_hist_df = pd.DataFrame(test_hist)
print(len(test_hist["action"]))

37687


In [32]:
test_hist_df

,current_step,balance,net_worth,tokens_held,token,current_price,reward,action,total_shares_sold,total_sales_value
0,48,219.827113,999.220607,9,1,86.599277,0.208342,0.802791,0,0.000000
1,49,45.807941,1001.957240,11,1,86.922664,0.477445,0.806853,0,0.000000
2,50,45.807941,996.437399,11,1,86.420860,-5.079954,0.804585,0,0.000000
3,51,45.807941,995.637673,11,1,86.348157,-4.615229,0.798631,0,0.000000
4,52,45.807941,999.708348,11,1,86.718219,-5.923298,0.817221,0,0.000000
...,...,...,...,...,...,...,...,...,...,...
37682,38162,0.000002,7307.904311,82265648,47,0.000089,3.212973,0.953930,937613456,22762.200006
37683,38163,0.000002,7374.045892,82265648,47,0.000090,-1.025220,0.951070,937613456,22762.200006
37684,38164,0.000002,7123.382463,82265648,47,0.000087,4.854645,0.948275,937613456,22762.200006
37685,38165,0.000002,7317.940720,82265648,47,0.000089,6.115366,0.943452,937613456,22762.200006


In [33]:
unique_values = test_hist_df['token'].unique()
unique_values

array([ 1,  5,  7, 26, 31, 32, 38, 41, 45, 47], dtype=int64)

## Plots

In [34]:
token = 47
plot_token_data(df = test_hist_df, token = token)
plot_close_by_asset(df= test_df, asset_id= token)

## Metrics

In [35]:
def evaluate_model(hist_df, test_df, initial_balance):
    """
    Evaluate the model's performance.

    Parameters:
    - hist_df: DataFrame containing the testing history.
    - test_df: DataFrame containing the test data.
    - initial_balance: Initial balance used in the environment.

    Returns:
    - report_df: DataFrame containing performance metrics per asset.
    - overall_metrics: Dictionary containing overall performance metrics.
    """

    # Ensure timestamps are in order
    hist_df = hist_df.sort_values('current_step').reset_index(drop=True)

    # List of assets
    assets = hist_df['token'].unique()

    # Initialize report DataFrame
    report = []

    for asset_id in assets:
        asset_hist = hist_df[hist_df['token'] == asset_id]
        asset_data = test_df[test_df['Asset_ID_encoded'] == asset_id]

        # Calculate total profit/loss
        final_net_worth = asset_hist['net_worth'].iloc[-1]
        total_profit = final_net_worth - initial_balance

        # Calculate ROI
        roi = (final_net_worth - initial_balance) / initial_balance * 100

        # Calculate Sharpe Ratio
        returns = asset_hist['net_worth'].pct_change().dropna()
        if returns.std() != 0:
            sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252)  # Assuming daily data
        else:
            sharpe_ratio = np.nan  # Undefined if no variance

        # Calculate Maximum Drawdown
        cumulative_returns = (1 + returns).cumprod()
        cumulative_max = cumulative_returns.cummax()
        drawdown = (cumulative_returns - cumulative_max) / cumulative_max
        max_drawdown = drawdown.min()

        # Calculate Win Rate
        trades = asset_hist[asset_hist['action'] != 0]
        wins = trades[trades['net_worth'].diff() > 0]
        win_rate = len(wins) / len(trades) * 100 if len(trades) > 0 else np.nan

        # Buy-and-Hold Strategy
        initial_price = asset_data['Close_orig'].iloc[0]
        final_price = asset_data['Close_orig'].iloc[-1]
        buy_and_hold_profit = (final_price - initial_price) * (initial_balance / initial_price)
        buy_and_hold_roi = (final_price - initial_price) / initial_price * 100

        # Ideal Strategy
        min_price = asset_data['Close_orig'].min()
        max_price = asset_data['Close_orig'].max()
        ideal_profit = (max_price - min_price) * (initial_balance / min_price)
        ideal_roi = (max_price - min_price) / min_price * 100

        # Collect metrics
        report.append({
            'Asset_ID': asset_id,
            'Total Profit': total_profit,
            'ROI (%)': roi,
            'Sharpe Ratio': sharpe_ratio,
            'Max Drawdown (%)': max_drawdown * 100,
            'Win Rate (%)': win_rate,
            'Buy-and-Hold Profit': buy_and_hold_profit,
            'Buy-and-Hold ROI (%)': buy_and_hold_roi,
            'Ideal Profit': ideal_profit,
            'Ideal ROI (%)': ideal_roi,
            'Asset Price Change (%)': (final_price - initial_price) / initial_price * 100,
        })

    # Create DataFrame from report
    report_df = pd.DataFrame(report)

    # Calculate overall averages for each column
    averages = report_df.mean(numeric_only=True)
    averages['Asset_ID'] = 'Average'  # Mark row as average

    # Append averages row to the DataFrame using pd.concat
    report_df = pd.concat([report_df, pd.DataFrame([averages])], ignore_index=True)

    # Calculate overall metrics
    overall_profit = report_df['Total Profit'].sum()
    overall_roi = (overall_profit / (initial_balance * len(assets))) * 100
    overall_sharpe = report_df['Sharpe Ratio'].mean()
    overall_win_rate = report_df['Win Rate (%)'].mean()
    overall_buy_and_hold_profit = report_df['Buy-and-Hold Profit'].sum()
    overall_buy_and_hold_roi = (overall_buy_and_hold_profit / (initial_balance * len(assets))) * 100

    overall_metrics = {
        'Total Profit': overall_profit,
        'ROI (%)': overall_roi,
        'Sharpe Ratio': overall_sharpe,
        'Win Rate (%)': overall_win_rate,
        'Buy-and-Hold Profit': overall_buy_and_hold_profit,
        'Buy-and-Hold ROI (%)': overall_buy_and_hold_roi,
    }

    return report_df, overall_metrics

- Asset_ID: Уникальный идентификатор актива (из столбца token), для которого рассчитываются метрики.

- Total Profit: Общий финансовый результат (прибыль или убыток) по данному активу. Рассчитывается как разница между конечной чистой стоимостью (net_worth) и начальным балансом (initial_balance).

- ROI (%): Доходность инвестиций (Return on Investment) в процентах. Показывает процентный прирост (или убыток) от начальной суммы баланса.

- Sharpe Ratio: Коэффициент Шарпа. Оценивает отношение доходности к риску (волатильности). Чем выше коэффициент Шарпа, тем лучше риск-корректированная доходность стратегии.

- Max Drawdown (%): Максимальная просадка в процентах. Это максимальное снижение стоимости актива от его исторического максимума. Отражает риски стратегии, связанные с падением стоимости.

- Win Rate (%): Процент прибыльных сделок. Это отношение количества прибыльных сделок к общему количеству сделок по активу, умноженное на 100.

- Buy-and-Hold Profit: Прибыль при стратегии "купить и держать". Показывает, сколько можно было бы заработать, если просто купить актив в начале и держать его до конца периода тестирования.

- Buy-and-Hold ROI (%): Доходность при стратегии "купить и держать". Процентный прирост от начальной цены актива, если его просто держать до конца периода.

- Ideal Profit: Идеальная прибыль. Это гипотетическая максимальная прибыль, которую можно было бы получить, если бы купили актив по минимальной цене и продали по максимальной цене за период.

- Ideal ROI (%): Идеальная доходность. Процентный прирост при идеальной стратегии, где покупка происходит по минимальной цене, а продажа — по максимальной.

- Asset Price Change (%): Изменение цены актива в процентах за период. Это процентное изменение цены от начальной до конечной за период тестирования.

In [36]:
report_df, overall_metrics = evaluate_model(test_hist_df, test_df, initial_balance = 1000)

C:\Users\McHomak\AppData\Local\Temp\ipykernel_19496\3864715686.py:85: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Average' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.



In [37]:
report_df

,Asset_ID,Total Profit,ROI (%),Sharpe Ratio,Max Drawdown (%),Win Rate (%),Buy-and-Hold Profit,Buy-and-Hold ROI (%),Ideal Profit,Ideal ROI (%),Asset Price Change (%)
0,1,939.969342,93.996934,0.411893,-34.070780,48.603498,926.933665,92.693366,1412.053634,141.205363,92.693366
1,5,364.927226,36.492723,0.241608,-32.864854,47.548045,-208.852052,-20.885205,1122.865439,112.286544,-20.885205
2,7,-479.054210,-47.905421,0.004479,-62.910924,49.953358,-654.146155,-65.414616,2912.288333,291.228833,-65.414616
3,26,1122.294361,112.229436,0.325964,-70.507021,46.969697,3956.937799,395.693780,4955.223881,495.522388,395.693780
4,31,-510.408273,-51.040827,-0.072689,-78.479560,50.603809,-426.135567,-42.613557,3343.885237,334.388524,-42.613557
5,32,0.000000,0.000000,NaN,0.000000,0.000000,118.150065,11.815006,241.530299,24.153030,11.815006
6,38,434.741789,43.474179,0.222708,-47.159709,50.456800,587.192841,58.719284,2286.449328,228.644933,58.719284
7,41,-82.857682,-8.285768,0.067254,-68.363160,48.261709,172.258836,17.225884,2660.366860,266.036686,17.225884
8,45,-683.135022,-68.313502,-0.149743,-82.401470,47.040280,-661.618848,-66.161885,4711.546925,471.154692,-66.161885
9,47,6595.834079,659.583408,0.505928,-85.874368,49.117024,12224.544937,1222.454494,41948.087432,4194.808743,1222.454494


In [38]:
test_hist_df.describe()

,current_step,balance,net_worth,tokens_held,token,current_price,reward,action,total_shares_sold,total_sales_value
count,37687.000000,3.768700e+04,37687.000000,3.768700e+04,37687.000000,3.768700e+04,37687.000000,37687.000000,3.768700e+04,37687.000000
mean,19107.920211,2.625764e+02,1243.464582,8.969051e+07,27.413670,2.379572e+02,-3.988189,0.613706,6.357363e+08,4839.338459
std,11015.726682,5.452087e+02,1174.308497,2.720127e+08,16.140535,7.059611e+02,2.837841,0.559579,1.807715e+09,6299.372818
min,48.000000,4.283292e-08,176.563460,0.000000e+00,1.000000,4.240000e-07,-42.565647,-0.998811,0.000000e+00,0.000000
25%,9565.500000,4.419555e-02,689.720666,1.100000e+01,7.000000,6.308311e-02,-5.449120,0.608295,0.000000e+00,0.000000
50%,19083.000000,2.442461e-01,984.215929,9.260000e+02,31.000000,4.668271e-01,-4.579173,0.869271,4.150000e+03,3205.669729
75%,28648.500000,4.818522e+01,1225.034785,3.906000e+03,41.000000,4.403854e+00,-3.102991,0.940867,6.086700e+04,7670.729174
max,38166.000000,7.800747e+03,9512.686836,1.465640e+09,47.000000,2.767008e+03,54.299301,0.999554,7.414954e+09,22762.200006


In [39]:
report_df.to_csv(Path["reports"](model_num, data_num, "test"), index= False)
test_hist_df.describe().to_csv(Path["reports"](model_num, data_num, "test_describe"), index= False)

In [40]:
overall_metrics

{'Total Profit': 8472.542771319353,
 'ROI (%)': 84.72542771319354,
 'Sharpe Ratio': 0.17304467598012002,
 'Win Rate (%)': 43.85542197639713,
 'Buy-and-Hold Profit': 17638.79207294396,
 'Buy-and-Hold ROI (%)': 176.3879207294396}